## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
weather_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
weather_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,52.48,96,100,1.88,light rain
1,1,Butaritari,KI,3.0707,172.7902,80.02,80,54,11.41,moderate rain
2,2,Georgetown,MY,5.4112,100.3354,82.33,88,20,2.30,few clouds
3,3,Churapcha,RU,62.0025,132.4325,-23.58,100,100,2.35,overcast clouds
4,4,Longyearbyen,SJ,78.2186,15.6401,17.44,67,0,3.44,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the weather_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                       (weather_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,80.02,80,54,11.41,moderate rain
2,2,Georgetown,MY,5.4112,100.3354,82.33,88,20,2.30,few clouds
8,8,Isangel,VU,-19.5500,169.2667,79.11,87,100,6.29,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,77.36,77,100,14.41,light rain
19,19,Kununurra,AU,-15.7667,128.7333,78.78,100,100,3.44,overcast clouds
23,23,Atuona,PF,-9.8000,-139.0333,78.35,70,24,17.74,few clouds
24,24,Port Blair,IN,11.6667,92.7500,75.67,76,34,7.63,scattered clouds
26,26,Arraial Do Cabo,BR,-22.9661,-42.0278,80.56,78,75,10.36,broken clouds
36,36,Busselton,AU,-33.6500,115.3333,79.05,42,63,8.70,broken clouds
37,37,Kavieng,PG,-2.5744,150.7967,80.26,80,100,5.82,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                202
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_df = preferred_cities_df.dropna()


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,80.02,moderate rain,3.0707,172.7902,
2,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
8,Isangel,VU,79.11,overcast clouds,-19.5500,169.2667,
14,Rikitea,PF,77.36,light rain,-23.1203,-134.9692,
19,Kununurra,AU,78.78,overcast clouds,-15.7667,128.7333,
23,Atuona,PF,78.35,few clouds,-9.8000,-139.0333,
24,Port Blair,IN,75.67,scattered clouds,11.6667,92.7500,
26,Arraial Do Cabo,BR,80.56,broken clouds,-22.9661,-42.0278,
36,Busselton,AU,79.05,broken clouds,-33.6500,115.3333,
37,Kavieng,PG,80.26,light rain,-2.5744,150.7967,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping.")
        

Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.
Hotel not found. Skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
# No missing hotel names, so I didn't need to do this step.
hotel_df.count()

City                   202
Country                202
Max Temp               202
Current Description    202
Lat                    202
Lng                    202
Hotel Name             202
dtype: int64

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))